<a href="https://colab.research.google.com/github/anyongho/Stockanalyzer/blob/main/%EC%9D%B8%EC%A7%80%EC%B6%94_%EA%B8%B0%EB%A7%90%EA%B3%BC%EC%A0%9C_%EC%9E%90%EB%A3%8C%EC%A4%80%EB%B9%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import yfinance as yf
import datetime as dt
import requests
from tqdm import tqdm

# STEP 1. S&P 500 심볼 크롤링
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get(url, headers=headers)
response.raise_for_status()

tables = pd.read_html(response.text)
sp500_table = tables[1] # <--- Changed from tables[0] to tables[1]
tickers = sp500_table["Symbol"].tolist()
tickers = [t.replace(".", "-") for t in tickers]

# ▲ 벤치마크 ^GSPC (S&P500 Index) 추가
benchmark_ticker = "^GSPC"
tickers = [benchmark_ticker] + tickers

# ▼ 3개월물 단기국채(미국 T-Bill) 대표 티커: ^IRX (CBOE 13 Week Treasury Bill Yield), 또는 SHY (1-3y ETF)
#     산식에 사용할 것 대비 가용 데이터 상태에 따라 교체 가능
rf_ticker = "^IRX"
tickers = [rf_ticker] + tickers

print(f"✅ 총 {len(tickers)}개 종목+지표 로드 완료")
print(tickers[:10])

# STEP 2. 5년치 데이터 다운로드 및 저장
start_date = (dt.datetime.now() - dt.timedelta(days=365 * 5)).strftime("%Y-%m-%d")
end_date = dt.datetime.now().strftime("%Y-%m-%d")
output_file = "SP500_AdjustedClose_5Y.xlsx"

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for ticker in tqdm(tickers, desc="Downloading Adjusted Close data"):
        try:
            df = yf.download(
                ticker,
                start=start_date,
                end=end_date,
                progress=False,
                auto_adjust=True,
            )

            # MultiIndex 처리 (필요시)
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel(1)
            # 조정종가만 추출 (단, ^IRX 등 일부 금리계열은 Close 사용)
            if "Adj Close" in df.columns:
                df = df[["Adj Close"]].copy()
                df.rename(columns={"Adj Close": "Adj Close"}, inplace=True)
            elif "Close" in df.columns:
                df = df[["Close"]].copy()
                df.rename(columns={"Close": "Adj Close"}, inplace=True)
            else:
                print(f"⚠️ {ticker}에 유효한 종가 컬럼 없음")
                continue
            df.reset_index(inplace=True)
            df.to_excel(writer, sheet_name=ticker[:31], index=False)
        except Exception as e:
            print(f"❌ 실패: {ticker} ({e})")

print(f"\n✅ 완료! Excel 파일 저장됨 → {output_file}")


/tmp/ipython-input-1499726811.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


✅ 총 505개 종목+지표 로드 완료
['^IRX', '^GSPC', 'MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES']



✅ 완료! Excel 파일 저장됨 → SP500_AdjustedClose_5Y.xlsx


In [ ]:
output_filename = 'sp500_companies.xlsx'
sp500_table.to_excel(output_filename, index=False)
print(f"✅ '{output_filename}' 파일로 저장되었습니다.")

✅ 'sp500_companies.xlsx' 파일로 저장되었습니다.
